`text2vec-transformers` is **only** available through Weaviate open-source. Here are options to select your desired model: 

1. [Pre-built transformers model containers](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#pre-built-images)

2. [Any model from Hugging Face Model Hub](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#option-2-use-any-publicly-available-hugging-face-model)

3. [Use any private or local PyTorch or Tensorflow transformer model](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#option-3-custom-build-with-a-private-or-local-model)

## Dependencies

In [ ]:
!pip install weaviate-client

## Connect to Weaviate

In [1]:
import weaviate
from weaviate.config import AdditionalConfig

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local(
    additional_config=AdditionalConfig(timeout=[900,1200])
)

client.is_ready()

True

## Create a collection
> Collection stores your data and vector embeddings.

In [3]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_transformers( # specify the vectorizer and model type you're using
        # pooling_strategy="masked_mean"
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

Successfully created collection: JeopardyQuestion.


## Import the Data

In [4]:
import requests, json
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
with jeopardy.batch.fixed_size(
    batch_size=2,
    # concurrent_requests=1
) as batch:
    for item in data:
        print("Adding", item)
        batch.add_object(item)

print("Import Complete")

if (len(jeopardy.batch.failed_objects) > 0):
    print("There were some errors")
    for fo in jeopardy.batch.failed_objects:
        print(fo)

Adding {'Category': 'SCIENCE', 'Question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'Answer': 'Liver'}
Adding {'Category': 'ANIMALS', 'Question': "It's the only living mammal in the order Proboseidea", 'Answer': 'Elephant'}
Adding {'Category': 'ANIMALS', 'Question': 'The gavial looks very much like a crocodile except for this bodily feature', 'Answer': 'the nose or snout'}
Adding {'Category': 'ANIMALS', 'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'Answer': 'Antelope'}
Adding {'Category': 'ANIMALS', 'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'Answer': 'the diamondback rattler'}
Adding {'Category': 'SCIENCE', 'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'Answer': 'species'}
Adding {'Category': 'SCIENCE', 'Question': 'A metal that is "ductile" can be pulled into this while cold & u

In [5]:
jeopardy = client.collections.get("JeopardyQuestion")
print(jeopardy.aggregate.over_all())

AggregateReturn(properties={}, total_count=10)


## Query Weaviate: Similarity Search (Text objects)

Similarity search options for text objects in **Weaviate**:

1. [near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#an-object)

3. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#a-vector)

### nearText Example

Find a `JeopardyQuestion` about "animals in movies". Limit it to only 4 responses.

In [6]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.near_text(
    query="african beasts",
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

ID: 49b803cc-4f3e-43eb-adf9-65184af90b98
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
} 

ID: e5fd5bc5-2d46-4efd-b551-2f4f53d600e7
Data: {
  "answer": "the nose or snout",
  "question": "The gavial looks very much like a crocodile except for this bodily feature",
  "category": "ANIMALS"
} 

ID: 8a7b9d88-1e3f-4fd2-b778-a29d309fd88a
Data: {
  "answer": "Elephant",
  "question": "It's the only living mammal in the order Proboseidea",
  "category": "ANIMALS"
} 

ID: efc2f4fb-857b-4976-a32b-ecc5b49f3538
Data: {
  "answer": "the diamondback rattler",
  "question": "Heaviest of all poisonous snakes is this North American rattlesnake",
  "category": "ANIMALS"
} 



Return vector embeddings.

In [7]:
response = jeopardy.query.near_text(
    query="african beasts",
    include_vector=True,
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2))
    print("Vector:", item.vector, "\n")

ID: 49b803cc-4f3e-43eb-adf9-65184af90b98
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
}
Vector: {'default': [-0.3713511824607849, -0.037091415375471115, -0.2973206639289856, 0.23234812915325165, 0.3163280487060547, -0.16909991204738617, -0.3601011037826538, 0.46147531270980835, 0.26754018664360046, 0.39349156618118286, -0.2829388976097107, -0.32842811942100525, -0.020972799509763718, -0.49931949377059937, -0.1661771982908249, -0.2833065688610077, -0.2244662195444107, 0.1135106161236763, 0.017157776281237602, -0.061033423990011215, -0.30129387974739075, -0.12509910762310028, -0.19521333277225494, 0.4077644348144531, 0.19974443316459656, -0.3287477195262909, 0.1403944492340088, 0.19677895307540894, 0.3159879446029663, 0.08474166691303253, -0.07856877148151398, -0.04837050661444664, -0.5164233446121216, 0.13331252336502075, -0.12397083640098572, 0.07254515588283539, -0.060297086834

Now, also request the `distance` for each returned item.

In [8]:
import weaviate.classes.query as wq

response = jeopardy.query.near_text(
    query="african beasts",
    return_metadata=wq.MetadataQuery(distance=True),
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Distance:", item.metadata.distance)
    print("Data:", item.properties, "\n")

ID: 49b803cc-4f3e-43eb-adf9-65184af90b98
Distance: 0.3930387496948242
Data: {'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'} 

ID: e5fd5bc5-2d46-4efd-b551-2f4f53d600e7
Distance: 0.4223722219467163
Data: {'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'} 

ID: 8a7b9d88-1e3f-4fd2-b778-a29d309fd88a
Distance: 0.44727039337158203
Data: {'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'} 

ID: efc2f4fb-857b-4976-a32b-ecc5b49f3538
Distance: 0.4639533758163452
Data: {'answer': 'the diamondback rattler', 'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'category': 'ANIMALS'} 



### nearObject Example

Search through the `JeopardyQuestion` class to find the top 4 objects closest to id `a1dd67f9-bfa7-45e1-b45e-26eb8c52e9a6`. (The id was taken from the query above)

In [10]:
response = jeopardy.query.near_object(
    near_object="efc2f4fb-857b-4976-a32b-ecc5b49f3538", # replace with your id of interest
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")

ID: efc2f4fb-857b-4976-a32b-ecc5b49f3538
Data: {'answer': 'the diamondback rattler', 'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'category': 'ANIMALS'} 

ID: e5fd5bc5-2d46-4efd-b551-2f4f53d600e7
Data: {'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'} 

ID: 41998f84-c7c2-48b0-95f2-6bd8d5b283e0
Data: {'answer': 'wire', 'question': 'A metal that is "ductile" can be pulled into this while cold & under pressure', 'category': 'SCIENCE'} 

ID: 49b803cc-4f3e-43eb-adf9-65184af90b98
Data: {'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'} 



### nearVector Example

Search through the `JeopardyQuestion` class to find the top 2 objects closest to the query vector `[-0.0125526935, -0.021168863, ... ]`

In [ ]:
response = jeopardy.query.near_vector(
    near_vector=[-0.0125526935, -0.021168863, ... ], # your vector object goes here
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")